In [1]:
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
import pickle
import os
from sklearn.linear_model import LogisticRegression
import time
import warnings
warnings.simplefilter(action='ignore')
warnings.filterwarnings(action='ignore')
from greedy_selection import GreedySelector

In [2]:
path = '/home/jovyan/Collection/Data/t_31_60/'

In [3]:
with open(path + 'train_big1', 'rb') as f:
    train = pickle.load(f)

In [4]:
train = train[train.columns[:15]]

In [5]:
train.drop(columns='target_1', inplace=True)

In [6]:
train

,cnum,dt,target_6,overdue_principal,overdue_amount,current_outstanding,total_overdue_interest,pastdue_share,mwb,frec_k_mwb,all_calls_1_mon,max_call_duration_1_mon,contact_calls_1_mon,contact_rate_calls_1_mon
0,00010067,2020-09-10,1.0,85627.18,118870.88,2043469.86,33243.70,0.058171,84.0,1.190476,54,181.0,11,0.203704
1,00010300,2020-01-16,0.0,39119.29,67277.42,1322366.69,28158.13,0.050877,121.0,43.801653,0,0.0,0,-1.000000
2,00010507,2020-08-28,1.0,12187.25,17093.88,112810.15,4906.63,0.151528,123.0,21.138211,150,276.0,7,0.046667
3,00010512,2020-08-20,0.0,291.73,306.08,138637.47,14.35,0.002208,122.0,6.557377,313,0.0,0,0.000000
4,00010640,2019-01-03,0.0,10926.40,17488.45,360523.16,6562.05,0.048509,56.0,14.285714,0,0.0,0,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27665,06015497,2020-02-28,0.0,574930.22,697238.41,8038027.96,122308.19,0.086742,13.0,23.076923,0,0.0,0,-1.000000
27666,06016444,2019-11-15,0.0,378690.42,549971.18,7401423.58,171280.76,0.074306,8.0,50.000000,0,0.0,0,-1.000000
27667,06017959,2020-05-15,0.0,2611291.94,2705493.07,0.00,94201.13,1.000000,13.0,15.384615,0,0.0,0,-1.000000
27668,06019279,2020-12-18,0.0,1218793.89,1522965.41,11576108.68,304171.52,0.131561,20.0,25.000000,0,0.0,0,-1.000000


In [7]:
features = train.drop(columns=['cnum', 'dt', 'target_6']).columns.tolist()

In [8]:
features

['overdue_principal',
 'overdue_amount',
 'current_outstanding',
 'total_overdue_interest',
 'pastdue_share',
 'mwb',
 'frec_k_mwb',
 'all_calls_1_mon',
 'max_call_duration_1_mon',
 'contact_calls_1_mon',
 'contact_rate_calls_1_mon']

In [9]:
model = LogisticRegression(random_state=84)

In [10]:
def my_metric(y_true, y_pred):
    return -1 + 2*roc_auc_score(y_true, y_pred)

In [12]:
new_model = GreedySelector(X=train[features + ['cnum']],
                           y=train['target_6'],
                           n_jobs=-1,
                           improvement=0.001,
                           group_field='cnum',
                            model=model,                           
                           name='presentation_model')

Всего признаков - 11


  0%|          | 0/11 [00:00<?, ?it/s]

Уже отобрано признаков - 0 



  0%|          | 0/11 [00:00<?, ?it/s]

 Отобранные признаки: ['overdue_principal'] 
 Метрика качества на обучающей выборке: 0.4196759244533106 
Уже отобрано признаков - 1 



  0%|          | 0/10 [00:00<?, ?it/s]

 Отобранные признаки: ['overdue_principal', 'current_outstanding'] 
 Метрика качества на обучающей выборке: 0.42137010546788023 
Уже отобрано признаков - 2 



  0%|          | 0/9 [00:00<?, ?it/s]

Метрика качества больше не улучшается. Все посчиталось. Поздравляю!
